In [ ]:
# importing libraries

import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
from transformers import *
import tokenizers
import os
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, TFBertModel, BertConfig
from sklearn.model_selection import StratifiedShuffleSplit
from transformers import BertTokenizer, TFBertForQuestionAnswering
from sklearn.model_selection import train_test_split
import gc
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import StratifiedKFold
from keras.utils import plot_model


In [ ]:
max_len = 168
train_mode = False

# Load the tokenizer
tokenizer = tokenizers.ByteLevelBPETokenizer(vocab_file = '../input/tf-roberta/vocab-roberta-base.json',
                             merges_file = '../input/tf-roberta/merges-roberta-base.txt',
                             add_prefix_space = True,
                             lowercase = True)

In [ ]:
# loading train data.
sentiment_id = {'positive': 1313, 'negative': 2430, 'neutral': 7974}
x_train = pd.read_csv('../input/tweet-sentiment-extraction/train.csv').fillna('')
x_train.head()

In [ ]:
# removing empty rows
x_train['text'].replace('', np.nan, inplace=True)
x_train.dropna(subset=['text'], inplace=True)
x_train.reset_index(drop=True, inplace=True)

## Data Preparation

In [ ]:
# https://www.kaggle.com/abhishek/bert-base-uncased-using-pytorch
# Example to explain operations in text_process function

tweet = 'WHY DO WE FALL? SO THAT WE CAN LEARN TO PICK OURSELVES BACK UP.'
selected_text = 'LEARN TO PICK OURSELVES BACK UP'
sentiment = 'positive'
print('text',tweet)
print('select_text:',selected_text)
print('sentiment:', sentiment)
# idx0 and idx1 and start and end indices of select_text in tweet.
idx0 = None
idx1 = None
st_len = len(selected_text)

# with roberta 'hello' and ' hello' are different hence we need to add a space for beginning of tweet.

tweet = " " + tweet


for i in range(len(tweet)):
    if(tweet[i:i+st_len]==selected_text):
        idx0 = i
        idx1 = i + st_len -1
        break

# char_tartgets is of length tweet, assign indices with select text =1 
char_targets = [0]*len(tweet)
for i in range(len(tweet)):
    if idx0 != None and idx1!=None:
        if i>=idx0 and i<=idx1:
            char_targets[i] = 1

print('char_targets:',char_targets)
# encoding tweet using tokenizer, it returns ids(token for each word) and offsets(span of each word)
tok_tweet = tokenizer.encode(tweet)

input_ids = tok_tweet.ids # word ids given by tokenizer stripping first[cls] and last token [sep]
offsets = tok_tweet.offsets # offsets of the tweet 

print('input_ids:',input_ids)
print('offsets:',offsets)
# start index and end index of tweet words with select_text
targets_index = []
for i, (off1,off2) in enumerate(offsets):
    if sum(char_targets[off1:off2])>0:
        targets_index.append(i)       
target_start = targets_index[0] 
target_end = targets_index[-1]

print('target_start:',target_start)
print('target_end:', target_end)

# creating ids, token_type_ids, mask into bert format, changing target_start and target_end accordingly.
ids = [0] + input_ids  + [2,2] + [sentiment_id[sentiment]] + [2]
token_type_ids = [0]*(len(ids))
mask = [1] * len(ids)
target_start+=1
target_end+=1
offsets = [(0,0)]*1 + offsets + [(0,0)]*4

# padding 
padding_length = max_len - len(ids)
if padding_length > 0:
    ids = ids + ([0] * padding_length)
    mask = mask + ([0] * padding_length)
    token_type_ids = token_type_ids + ([0] * padding_length)
    offsets = offsets + ([(0, 0)] * padding_length)


In [ ]:
# https://www.kaggle.com/abhishek/bert-base-uncased-using-pytorch

def text_process(tweet, sentiment, tokenizer, max_len, selected_text=None):
    
    """
    inputs:
    tweets: text 
    sentiment: sentiment of the tweet
    tokenizer: tokenizer
    max_len: max length of ids, mask and token_type_ids (inputs of bert)
    selected_text: selected_text (optional)
    
    operation:
    
    Given inputs it calculates  ids, mask , token_type_ids, offsets and target_start and target_end.
    
    outputs:
    dictionary with keys as below,
    ids: input tokens for roberta in format as [0] <text tokens> [2] [2] <sentiment> [2]
    mask: array with length as max_len and has 1's in the indices of text and zeros elsewhere.
    token_type_ids: all 0s , size max_len
    target_start,target_end: begin and end of select_text (returned only when select_text is given)
    offsets: offsets of text 
    
    """
    
    if selected_text!=None:
        
        tweet = " " + tweet
        idx0 = None
        idx1 = None
        st_len = len(selected_text)
        for i in range(len(tweet)):
            if(tweet[i:i+st_len]==selected_text):
                idx0 = i
                idx1 = i + st_len -1
                break

        char_targets = [0]*len(tweet)

        for i in range(len(tweet)):
            if idx0 != None and idx1!=None:
                if i>=idx0 and i<=idx1:
                    char_targets[i] = 1

        tok_tweet = tokenizer.encode(tweet)

        input_ids = tok_tweet.ids
        offsets = tok_tweet.offsets
        
        targets_index = []

        for i, (off1,off2) in enumerate(offsets):
            if sum(char_targets[off1:off2])>0:
                targets_index.append(i)

        target_start = targets_index[0] 
        target_end = targets_index[-1]
        
        
        ids = [0] + input_ids  + [2,2] + [sentiment_id[sentiment]] + [2]
        token_type_ids = [0]*(len(ids))
        mask = [1] * len(ids)
        target_start+=1
        target_end+=1
        offsets = [(0,0)]*1 + offsets + [(0,0)]*4

        padding_length = max_len - len(ids)
        if padding_length > 0:
            ids = ids + ([0] * padding_length)
            mask = mask + ([0] * padding_length)
            token_type_ids = token_type_ids + ([0] * padding_length)
            offsets = offsets + ([(0, 0)] * padding_length)
            
        return {
            
            'ids': ids,
            'token_type_ids':token_type_ids,
            'mask':mask,
            'target_start':target_start,
            'target_end':target_end,
            'offsets':offsets
        }
    else:
        
        tweet = " " + tweet
        tok_tweet = tokenizer.encode(tweet)
        
        input_ids = tok_tweet.ids
        offsets = tok_tweet.offsets

        ids = [0] + input_ids  + [2,2] + [sentiment_id[sentiment]] + [2]
        token_type_ids = [0]*(len(ids))
        mask = [1] * len(ids)
        offsets = [(0,0)]*1 + offsets + [(0,0)]*4

        padding_length = max_len - len(ids)
        if padding_length > 0:
            ids = ids + ([0] * padding_length)
            mask = mask + ([0] * padding_length)
            token_type_ids = token_type_ids + ([0] * padding_length)
            offsets = offsets + ([(0, 0)] * padding_length)
            
        return {
            
            'ids': ids,
            'token_type_ids':token_type_ids,
            'mask':mask,
            'offsets':offsets
        }



In [ ]:
# creating ids,type_ids,mask , start_index and end_index for train set.
rows = x_train.shape[0]

train_ids = np.zeros((rows,max_len), dtype = 'int32')
train_mask = np.zeros((rows,max_len), dtype = 'int32')
train_type_ids = np.zeros((rows,max_len), dtype = 'int32')
train_start_idx = np.zeros((rows,max_len), dtype = 'int32')
train_end_idx = np.zeros((rows,max_len), dtype = 'int32')

for i in range(x_train.shape[0]):
    
    encoding = text_process(x_train.loc[i,'text'], x_train.loc[i,'sentiment'], tokenizer, max_len,x_train.loc[i,'selected_text'] )
    
    train_ids[i] = encoding['ids']
    train_start_idx[i,encoding['target_start']] = 1
    train_end_idx[i, encoding['target_end']] = 1
    
    train_type_ids[i] = encoding['token_type_ids']
    train_mask[i] = encoding['mask']

In [ ]:
# checking

i = 10

encoding = text_process(x_train.loc[i,'text'], x_train.loc[i,'sentiment'], tokenizer, max_len,x_train.loc[i,'selected_text'] )

train_ids[i] = encoding['ids']


train_start_idx[i,encoding['target_start']] = 1
train_end_idx[i, encoding['target_end']] = 1

train_type_ids[i] = encoding['token_type_ids']
train_mask[i] = encoding['mask']

tweet = x_train.loc[i,'text']
select_text = x_train.loc[i,'selected_text']

target_start = np.argmax(train_start_idx[i,])
target_end = np.argmax(train_end_idx[i, ])

offsets = encoding['offsets']


print('tweet:',tweet)
print('selected_text:',select_text)
print(tweet[offsets[target_start][0]:offsets[target_end][-1]])

## Modelling

In [ ]:
# Metric
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    
    if (len(a)==0) & (len(b)==0): 
        return 0.5
    
    c = a.intersection(b)
    
    return float(len(c)) / (len(a) + len(b) - len(c))

In [ ]:
# Config
config = RobertaConfig.from_pretrained('../input/tf-roberta/config-roberta-base.json')
print(config)

In [ ]:
# https://www.kaggle.com/c/tweet-sentiment-extraction/discussion/143281
def build_model():
    # Create Model
       
    ids = tf.keras.layers.Input((max_len,), dtype=tf.int32)
    att = tf.keras.layers.Input((max_len,), dtype=tf.int32)
    tok = tf.keras.layers.Input((max_len,), dtype=tf.int32)

    roberta = TFRobertaForQuestionAnswering.from_pretrained('../input/tf-roberta/pretrained-roberta-base.h5', config = config)
    x = roberta(ids, attention_mask = att, token_type_ids = tok)

    x1 = tf.keras.layers.Dropout(0.3)(x[0]) 
    x1 = tf.keras.layers.Activation('softmax')(x1)

    x2 = tf.keras.layers.Dropout(0.3)(x[1]) 
    x2 = tf.keras.layers.Activation('softmax')(x2)

    model = tf.keras.models.Model(inputs = [ids, att, tok], outputs=[x1, x2])
    optimizer = tf.keras.optimizers.Adam(learning_rate = 3e-5)

    model.compile(loss= 'categorical_crossentropy', optimizer=optimizer)
    return model

In [ ]:
def lr_scheduler(epoch):
    return 3e-5 * 0.2**epoch

In [ ]:

model = build_model()

model.summary()

In [ ]:
# plotting model
plot_model(model, show_shapes = True)


In [ ]:
# clearing space
gc.collect()

In [ ]:
# Training or loading trained model for predictions.

# https://www.kaggle.com/cdeotte/tensorflow-roberta-0-705
fold_start = np.zeros((train_ids.shape[0],max_len))
fold_end = np.zeros((train_ids.shape[0],max_len))

# will train 5 models and combine(avg) their output to get final predictions.
n_splits=5
kfold = StratifiedKFold(n_splits=n_splits,shuffle=True,random_state=42) 

for fold,(train_id,val_id) in enumerate(kfold.split(train_ids,x_train.sentiment.values)):

    model = build_model()
    
    # callbacks
    checkpoint = tf.keras.callbacks.ModelCheckpoint('roberta-%i.h5'%(fold), monitor = 'val_loss', verbose = 1, save_best_only = True,
                                                          save_weights_only = True, mode = 'auto', save_freq = 'epoch')
    reduce_lr_cb = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)

    if train_mode:

        model.fit([train_ids[train_id,], train_mask[train_id,], train_type_ids[train_id,]], [train_start_idx[train_id,], train_end_idx[train_id,]], 
                          epochs = 4, 
                          batch_size = 16, 
                          verbose = True, 
                          callbacks = [checkpoint,reduce_lr_cb],
                  validation_data = ([train_ids[val_id,], train_mask[val_id,], train_type_ids[val_id,]], [train_start_idx[val_id,], train_end_idx[val_id,]]),shuffle = True)
        
        model.load_weights('roberta-%i.h5'%(fold)) # loads best model.
        
    else:

        model.load_weights(f'../input/roberta-folds/roberta-{fold}.h5')
    # predictions on val data.
    fold_start[val_id],fold_end[val_id] = model.predict([train_ids[val_id,],train_mask[val_id,],train_type_ids[val_id,]],verbose = 1)
    
    # score on val_data.
    score = 0
    for k in val_id:

        if(x_train.loc[k,'sentiment']!='neutral'):

            
            encoding = text_process(x_train.loc[k,'text'], x_train.loc[k,'sentiment'], tokenizer, max_len)
            offsets = encoding['offsets']

            targets_start = np.argmax(fold_start[k,])
            targets_end = np.argmax(fold_end[k,])
            if targets_start<=targets_end:
                pred = x_train.loc[k,'text'][offsets[targets_start][0]:offsets[targets_end][-1]]
                score+=jaccard(x_train.loc[k,'selected_text'], pred)
            else:
                score+=jaccard(x_train.loc[k,'selected_text'], x_train.loc[k,'text'])

        else:
            score+=jaccard(x_train.loc[k,'selected_text'], x_train.loc[k,'text'])

    score=score/len(val_id) 
    print("fold",fold, "jaccard_score:",score)

## Test Predictions

In [ ]:
# reading test set.
x_test = pd.read_csv('../input/tweet-sentiment-extraction/test.csv')
rows = x_test.shape[0]
preds_start = np.zeros((rows,max_len))
preds_end = np.zeros((rows,max_len))

In [ ]:
# # creating ids,type_ids,mask , start_index and end_index for test set.

rows = x_test.shape[0]

test_ids = np.zeros((rows,max_len), dtype = 'int32')
test_mask = np.zeros((rows,max_len), dtype = 'int32')
test_type_ids = np.zeros((rows,max_len), dtype = 'int32')
test_start_idx = np.zeros((rows,max_len), dtype = 'int32')
test_end_idx = np.zeros((rows,max_len), dtype = 'int32')

for i in range(x_test.shape[0]):
    
    encoding = text_process(x_test.loc[i,'text'], x_test.loc[i,'sentiment'], tokenizer, max_len )
    test_ids[i] = encoding['ids']
    test_type_ids[i] = encoding['token_type_ids']
    test_mask[i] = encoding['mask']


In [ ]:
for i in range(n_splits):
    model.load_weights(f'../input/roberta-folds/roberta-{i}.h5')
    preds = model.predict([test_ids, test_mask, test_type_ids], verbose = True)
    preds_start += preds[0]/n_splits
    preds_end += preds[1] /n_splits

In [ ]:
submission = pd.read_csv('../input/tweet-sentiment-extraction/sample_submission.csv')

In [ ]:

for k in range(x_test.shape[0]):
    
    if x_test.loc[k,'sentiment']!='neutral':
        
        encoding = text_process(x_test.loc[k,'text'], x_test.loc[k,'sentiment'], tokenizer, max_len)
        offsets = encoding['offsets']
        targets_start = np.argmax(preds_start[k,])
        targets_end = np.argmax(preds_end[k,])
        
        if targets_start<=targets_end:
            pred = x_test.loc[k,'text'][offsets[targets_start][0]:offsets[targets_end][-1]]
            submission.loc[k,'selected_text'] = pred
        else:
            submission.loc[k,'selected_text'] = x_test.loc[k,'text']

    else:
        submission.loc[k,'selected_text'] = x_test.loc[k,'text']


In [ ]:
submission.to_csv('submission.csv', index = False)